In [1]:
import numpy as np
from scipy import optimize

In [2]:
E = 1.5e-4  # [m] -> Rugosidade da parede interna do duto

Din = 1.22 # [m] -> Diâmetro interno do duto

re = 1.7e+5 # n.º de reynolds para o fluido em questão

rho=998 # [kg/m3] -> Densidade do fluido (agua)

mu=0.001 # [Pa/s] -> Viscosidade dinamica do fluido (agua)

v=1.0 # [mm/s] -> Velocidade do escoamento

Re=rho*Din*v/mu # [] -> Numero de Reynolds
# (função) fator de atrito
start = 1
end = 100
spaces = 100
tolerancia = 1.00e-08
f0 = 0.009

##### Equação de Colebroke:

In [3]:
def colebroke(f):
    return 1 / (f ** 0.5) + 2.0 * np.log10(E / (3.7 * Din) + 2.51 / (Re * f ** 0.5))

In [4]:
def colebroke_derivada(f):
    return -0.5 * f ** (-1.5) - 2.51 * f ** (-1.5) / (Re * (2.51 * f ** (-0.5) / Re + 0.27027027027027 * E / Din) * np.log(10))

In [5]:
function, results = optimize.newton(colebroke, f0, colebroke_derivada, full_output=True)

In [6]:
function

0.013564789886192896

In [7]:
results

      converged: True
           flag: 'converged'
 function_calls: 10
     iterations: 5
           root: 0.013564789886192896